<a href="https://colab.research.google.com/github/Charmzyy/laravelgittutorial/blob/main/nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk -q


In [2]:
import nltk 

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
!pip install datasets -q

     |████████████████████████████████| 441 kB 21.1 MB/s 
     |████████████████████████████████| 95 kB 5.1 MB/s 
     |████████████████████████████████| 163 kB 43.4 MB/s 
     |████████████████████████████████| 212 kB 56.2 MB/s 
     |████████████████████████████████| 115 kB 63.4 MB/s 
     |████████████████████████████████| 127 kB 46.0 MB/s 
     |████████████████████████████████| 115 kB 54.5 MB/s 


In [4]:
from datasets import load_dataset

dataset = load_dataset("tweet_eval", "emoji")

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/45000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset tweet_eval downloaded and prepared to /root/.cache/huggingface/datasets/tweet_eval/emoji/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 5000
    })
})

In [6]:
texts=dataset['validation']['text']
labels=dataset['validation']['label']

print(len(texts),len(labels))

5000 5000


In [7]:
import pandas as pd

df = pd.DataFrame()

In [8]:
df['texts'] = texts
df['labels'] = labels
df


,texts,labels
0,A little throwback with my favourite person @ ...,0
1,glam on @user yesterday for #kcon makeup using...,7
2,Democracy Plaza in the wake of a stunning outc...,11
3,Then &amp; Now. VILO @ Walt Disney Magic Kingdom,0
4,Who never... @ A Galaxy Far Far Away,2
...,...,...
4995,don't know what I'd do without you @ Keene Sta...,0
4996,Senior night with my little Bailey !! So proud...,3
4997,Real friends or labeled as family! #BrotherMan...,6
4998,It makes me so happy meet people wearing hats ...,3


In [10]:
df['labels'].value_counts()

0     1056
1      521
2      504
3      308
4      243
5      238
6      204
7      199
8      177
9      171
10     159
13     153
15     148
11     143
16     137
19     131
14     129
12     129
18     129
17     121
Name: labels, dtype: int64

In [ ]:
df = df[df['labels'].isin([1,2])]
df.head()

In [12]:
df['labels'].value_counts()

1    521
2    504
Name: labels, dtype: int64

In [20]:
#text pre processing
print(df['texts'].iloc[2])

livinginparadise  ramada plaza beach resort


In [ ]:
df['texts'] = df['texts'].str.replace("@", "")
df['texts'] = df['texts'].str.replace("#", "")
df['texts'] = df['texts'].str.replace("\d+", "")
df['texts'] = df['texts'].str.lower()

In [21]:
df

,texts,labels
4,who never... a galaxy far far away,2
5,dinner in fla tonight // pan-seared salmon ove...,1
12,livinginparadise ramada plaza beach resort,1
18,because more days echo beach,1
21,when the sun hit you but you just still cute,1
...,...,...
4963,my son last night poopybaby alex vargas' hear...,2
4966,user i am so jealous you look like you're havi...,1
4969,tomycity spacemountain papabearandi straight u...,2
4993,these are a few of my favorite things newmexic...,1


In [22]:
import itertools  #iteration tools

vocab = list(itertools.chain.from_iterable(df['texts']))
vocab =list(set(vocab))
len(vocab)

57

In [23]:
#feature  extraction

#BoW

from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer()
bow.fit(df['texts'])


CountVectorizer()

In [24]:
bow = bow.transform(df['texts'])
print(bow.shape)

(1025, 3675)


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(2, 2))
tfidf.fit(df['texts'])
tfidf = tfidf.transform(df['texts'])
print(tfidf.shape)


(1025, 8218)


In [27]:
from sklearn.model_selection import train_test_split as ts


Xt, Xts, yt, yts = ts(tfidf, df['labels'])


from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score as acc


knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(Xt,yt)
knn_pred = knn.predict(Xts)
print(acc(knn_pred, yts))

nb = MultinomialNB()
nb.fit(Xt,yt)
nb_pred = nb.predict(Xts)
print(acc(nb_pred , yts))

0.5875486381322957
0.6303501945525292
